In [1]:
import requests
import pandas as pd
import joblib
import time

# Load the trained model and scaler
model = joblib.load(r'C:\Users\JANU\nsvmrbf.pkl')  # Path to your model file
scaler = joblib.load(r'C:\Users\JANU\nscal.pkl')  # Path to your scaler file

# Map the prediction values to activities
prediction_mapping = {
    1: "fall",
    3: "jog",
    4: "walk",
    5: "std",
    6: "run",
    7: "jump"
}

# Fetch data from Firebase Realtime Database
def fetch_data_from_firebase(firebase_url, data_path):
    try:
        response = requests.get(firebase_url + data_path + '.json')
        response.raise_for_status()  # Raise an error for HTTP codes 4xx/5xx
        data = response.json()
        return data
    except requests.RequestException as e:
        print(f"Error fetching data from Firebase: {e}")
        return {}

def preprocess_and_predict(data):
    try:
        # Ensure all necessary features are present
        required_features = ['AccelX', 'AccelY', 'AccelZ', 'GyroX', 'GyroY', 'GyroZ']
        if not all(feature in data for feature in required_features):
            print("Missing one or more required features.")
            return None

        # Remove 'Label' field if present, since it's not required for prediction
        if 'Label' in data:
            del data['Label']

        # Convert values to float
        data = {k: float(v) for k, v in data.items()}

        # Convert data to DataFrame
        df = pd.DataFrame([data])  # Wrap data in a list to handle single entries

        # Select only the features that the model was trained on
        df = df[required_features]

        # Scale the features
        X = scaler.transform(df)

        # Make prediction
        predictions = model.predict(X)

        return int(predictions[0])  # Convert prediction to standard Python int

    except Exception as e:
        print(f"Error during preprocessing or prediction: {e}")
        return None

# Send predictions to Firebase
def send_predictions_to_firebase(activity, prediction_value, firebase_url, result_paths):
    try:
        # Create a dictionary with activity and prediction value
        result_data = {
            'predicate': activity,          # The corresponding activity name (Fall, Jog, etc.)
            'prediction': prediction_value,  # The raw prediction value
            'steps':"333",
            'heartRate':"85"
            
        }
        
        # Send the data to each path in the result_paths list
        for path in result_paths:
            full_url = firebase_url + path + '.json'
            response = requests.put(full_url, json=result_data)
            response.raise_for_status()
            print(f"Prediction successfully sent to Firebase at {path}")
    except requests.RequestException as e:
        print(f"Error sending activity to Firebase: {e}")



# Main loop
firebase_url = "https://harsha-3a181-default-rtdb.firebaseio.com/"  # Your Firebase Realtime Database URL
data_path = "/Falldetect"  # Adjust the path to match where your data is stored in Firebase
result_paths = [
    "/predictions",        # Path to send activity to Firebase
    "/users/raju/stats"    # Another path to send activity to Firebase
]

while True:  # Infinite loop
    print("Fetching new data...")
    new_data = fetch_data_from_firebase(firebase_url, data_path)
    if new_data:
        print("Data received:", new_data)
        prediction = preprocess_and_predict(new_data)
        if prediction is not None:
            # Map the prediction to the corresponding activity
            activity = prediction_mapping.get(prediction, "Unknown Activity")
            print(f"Prediction made: {activity} (Value: {prediction})")
            
            # Send both the activity name and the prediction value to Firebase
            send_predictions_to_firebase(activity, prediction, firebase_url, result_paths)
    else:
        print("No new data found.")
    time.sleep(10)  # Wait for 10 seconds before the next iteration


Fetching new data...
Data received: {'AccelX': '1.22', 'AccelY': '-146.89', 'AccelZ': '16.55', 'GyroX': '-1.30', 'GyroY': '-33.72', 'GyroZ': '-1.02', 'HeartRate': 0, 'SPO2': 0}
Prediction made: fall (Value: 1)
Prediction successfully sent to Firebase at /predictions
Prediction successfully sent to Firebase at /users/raju/stats
Fetching new data...
Data received: {'AccelX': '1.22', 'AccelY': '-146.89', 'AccelZ': '16.55', 'GyroX': '-1.30', 'GyroY': '-33.72', 'GyroZ': '-1.02', 'HeartRate': 0, 'SPO2': 0}
Prediction made: fall (Value: 1)
Prediction successfully sent to Firebase at /predictions
Prediction successfully sent to Firebase at /users/raju/stats
Fetching new data...
Data received: {'AccelX': '1.22', 'AccelY': '-146.89', 'AccelZ': '16.55', 'GyroX': '-1.30', 'GyroY': '-33.72', 'GyroZ': '-1.02', 'HeartRate': 0, 'SPO2': 0}
Prediction made: fall (Value: 1)
Prediction successfully sent to Firebase at /predictions
Prediction successfully sent to Firebase at /users/raju/stats
Fetching new 

KeyboardInterrupt: 